<a href="https://colab.research.google.com/github/mahendra-gehlot/CP260/blob/main/CP260_Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAB 5:

This is a simplifed setting of Lab 3.

The map is M(Rows x Cols)

The rat's motion is restricted to be in 
one dimension along the columns.

Each time step, the rat advances randomly to another column and makes a range measurement along the rows of that colum.

It gets back:
zRow : the index from which reflection occurs
zVal : Value of the feature

It needs to build the map and estimate its pose.

NOTE: 
Marks of LAB 3 are reduced to 15

**Total Marks 90**

[5] (a) Write equations for update of log odds for each map grid

[10] (b) Write your formula or pseudo code for likelihood calculation of each 
particle for each measurement

[60] (c) Fill in the logUpdate function and show the results

[10] (d) Find the error w.r.t number of particles.

[10] (e) Lab 3 results


The report can be found by following link-

https://indianinstituteofscience-my.sharepoint.com/:w:/g/personal/mahendrag_iisc_ac_in/EfebkxucCc1DilsOHL6yV3IBxgZEBLBeG04_dSxwdCt-VA?e=DwRJkI


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
#rats functions

def measure (
        rL,    #layout
        col,   #column on which to make a measurement
        rng_m, #measurement noise generator
        valStdErr, #std  error for value reading
        rNoiseDist,    #probability for choice of error in range. its size is 2*lim+1, where noise is upto +- lim
        ptm,    #probablity of true measurement 
        ):
    
    ''' Return a range hit with a noise of +- as per pr vector
        Modulate it further with probability pc['fp', 'fn', 'tm'] to
        override the true measurement (tm) with a false positive (from 0 to certain distance of target)
        or a false negative (no hit)
    '''

    lim = int((rNoiseDist.shape[0]-1)/2)

    
    isTrueMeasurement = rng_m['ch'].choice([True, False],p=[ptm,1-ptm])
    zRow = rL.shape[0] #default is out of range
    zVal = 0
    
    if isTrueMeasurement :
        zRow = np.argmax(rL[:,col])
        if rL[zRow,col] != 0 : #there is a feature here - so add noise
            zVal = rL[zRow,col] + rng_m['val'].normal(0,valStdErr) 
            zRow += rng_m['tm'].choice(list(range(-lim,lim+1)), p=rNoiseDist)
            if zRow >= rL.shape[0] :
                zRow = rL.shape[0]
            elif zRow < 0 :
                zRow = 0
        else: #no feature here - set range measurement to out of range
            zRow = rL.shape[0]
    #end if isTrueMeasurement
    return zRow, zVal
#end def measure

def move (
        x,     #current position
        step,  #step to take
        pDist, #triingle distribution
        rng_m, #noise generator
        nCols, #number of columns
        ):
    '''motion of robot'''
    
    lim = int((pDist.shape[0]-1)/2)
    if (2*lim+1 != pDist.shape[0]) :
        sys.exit("Error: pDist shape should be an odd integer")

    x += step + rng_m.choice(list(range(-lim,lim+1)), p=pDist) 
    
    return int(x % nCols)
#move


def sMove (
        x,     #current position
        step,  #step to take
        pDist, #triingle distribution
        rng_pm, #noise generator
        nCols, #number of columns
        ):
    '''motion model for particle'''
    lim = int((pDist.shape[0]-1)/2)
    if (2*lim+1 != pDist.shape[0]) :
        sys.exit("Error: pDist shape should be an odd integer")

    x += step + rng_pm.choice(list(range(-lim,lim+1)), p=pDist) 
    
    return int(x % nCols)

#sMove



In [ ]:
#Utility Functions

def buildMap (
        occList,  #list of tuples:x,y,v
        rL        #layout
        ):
    for (x,y,v) in occList:
        rL[x,y] = v
#end buildMap

def triangle (
        pCenter,    #probability of center
        base,       #odd positive the base of triangle
        ):
# this is a triangle probability distribution

    if  base %2 != 1 :
        print("ERROR: base needs to be odd")
    lim  = int ((base-1)/2)
    delta = (1.0-pCenter)/((lim+1)*lim)
    pDist = np.zeros(2*lim+1)
    pDist[lim] = pCenter
    for i in range(1,lim+1):
        pDist[lim-i] = (lim-i+1)*delta
        pDist[lim+i] = (lim-i+1)*delta
    return pDist
#end triangle

def getMap (
        i
        ):
    map = np.zeros(MAP_SIZE)
    map = np.greater_equal(pMap[i,LODD],0).astype(int)*pMap[i,VAL]
    map = map.astype(int)
    return(map)
#end getMap

#%%
def countErr (
        h1, #count array 
        i1, #index
        h2, #second count array
        i2   #second index
        ):
    NCOLS = h1.shape[1]
    err = 0
    for i in range(NCOLS):
        d1 = (i1 + i) % NCOLS       
        d2 = (i2 + i) % NCOLS       
        err += np.sum(np.abs(h1[:,d1]-h2[:,d2]))
    return err
#end countErr


def sir (
    pPose,
    pMap,
    rngSir
):
  '''Sample Importance Resampling'''
  NPARTICLES = pPose.shape[0]
  new_pPose = np.zeros(pPose.shape)
  new_pMap = np.zeros(pMap.shape)
  
  q = rngSir.random()*1.0/NPARTICLES
  c = pPose[0,WT]
  k = 0
  for j in range(NPARTICLES):
    U = q+j/NPARTICLES
    while U > c :
      k = k + 1
      c = c + pPose[k,WT]
    new_pPose[j] = pPose[k]
    new_pMap[j]  = pMap[k]
    new_pPose[j,LOGWT] = 0

  return new_pPose, new_pMap
#end sir


In [ ]:
    # if zRow < MAP_ROWS:
    #   l_zVal = v[zRow][col];
    # else:
    #   l_zVal = 0
    # #Storing features values
    # 

    # # print("M",m)
    # # print("c",c)


    # # if np.sum(c) != 0:
    # #   if zRow < MAP_ROWS:
    # #     logLkl = c[zRow][col]/np.sum(c) + 5 if (v[zRow][col] <= l_zVal + valStdErr or v[zRow][col] >= l_zVal - valStdErr) else 2
    # #   else:
    # #     logLkl = 0

In [ ]:
def logOddsUpdate (
        zRow, #measurement value of hit
        zVal, #value measurement
        valStdErr, #std error for value measurement (assume normal distribution)
        pPose, #particle pose
        pMap, # particle map data structures
        ptm, #prob of true measurement
        pErr, #probabiliy of error in true measurement
        ):
    '''update the log odds of the map column m[:,col]'''
   
    m = pMap[LODD]
    v = pMap[VAL]
    c = pMap[CNT]
    col = pPose
    #calculate likelihood first 
    if zRow < MAP_ROWS:
      m[0:zRow,col] += np.log(pErr/ptm)
      m[zRow,col] += np.log(ptm/pErr)
    else:
      m[:,col] += np.log(pErr/ptm)
      
    
    logLkl = -10

    if zRow < MAP_ROWS:
      if c[zRow,col] != 0 and v[zRow,col] != 0:
        logLkl = (c[zRow,col]/np.sum(c))*((-(v[zRow,col]-zVal)**2)/(valStdErr**2))
        if c[zRow,col] == 0:
          logLkl = 1
        c[zRow,col] += 1
      else:
        c[zRow,col] += 1
        if v[zRow,col] == 0:
          v[zRow,col] = zVal
        else:
          v[zRow,col] = zVal if (zVal + valStdErr >= v[zRow,col] or zVal - valStdErr <= v[zRow,col]) else v[zRow,col]
          v[:zRow,col] = 0
    else:
      c[zRow,col] += 1
    
    
    return logLkl        
#end logOddsUpdate


In [ ]:
MAP_ROWS= 10
MAP_COLS = 10
MAP_SIZE = (MAP_ROWS,MAP_COLS)
NPARTICLES = 100
NSTEPS = 500
noisyMotion = False

realLayout = np.zeros(MAP_SIZE).astype(int) #occupant value

#noise source for particles
rng_p = np.random.default_rng(222)
rng_pm = np.random.default_rng(224)

#noise source for motion velocity
rng_v = np.random.default_rng(234)

#noise source for Sample Importance Resampling
rngSir = np.random.default_rng(44)

#noise sources for measurement
rng_m = {'fp':np.random.default_rng(123) ,   #rng for false positive error
         'tm':np.random.default_rng(345),    #rng for true measurement's noise error
         'ch':np.random.default_rng(567),    #rng for choice of noise (true positive, false positive, false negative) error
         'val':np.random.default_rng(444)    #rng for value error 
         }
valStdErr = 1  #standard deviation for value error (assume gaussian with mean 0)


pPose = np.zeros((NPARTICLES,3))  #particle pose and logwt
#indices for the array
POSE = 0
LOGWT = 1
WT = 2

pMap = np.zeros((NPARTICLES,3,MAP_ROWS+1,MAP_COLS))
#add an extra row to keep track of out range hits

#indices for the array
LODD = 0
VAL = 1
CNT = 2

pCntErr = np.zeros(NPARTICLES)

ptm = 0.95   #probability of true measurement


#range error noise as a triangle distribution
pErr = 0.05  #probability of an error in a range hit measurement
rBase = 3 #base of triangle distribition (-1, 0, 1)
rNoiseDist = triangle(1-pErr, rBase) #pmf for range noise error distribution

#motion noise as a triangle distribution
pNoMotionNoise = 0.9  #probability of no motion noise for real robot
mBase = 3  #base of triangle distribution (-1, 0 , 1)
mNoiseDist = triangle(pNoMotionNoise,mBase)
pmNoMotionNoise = 0.9  #probability of no motion noise for particles
pmBase = 3  #base of triangle distribution (-1, 0 , 1)
pmNoiseDist = triangle(pmNoMotionNoise,pmBase)

occList = [
    (2,0, 100),
    (4,2,110),
    (2,3, 120),
    (4,4, 140),
    (8,8,100)    
    ]             

buildMap (occList, realLayout)

#print(realLayout)
realMap = np.greater(realLayout,0).astype(int)

#histogram to keep track of counts for hits to all cells in the map
#number of rows is one more to keep track of out of range measurements
hist = np.zeros((MAP_ROWS+1, MAP_COLS)) 

#pose of the robot across the time steps
rPose = np.zeros(NSTEPS).astype(int)

mapError = np.zeros(NSTEPS)
v = 1.0
for t in range(1,NSTEPS):
    #update the robot's pose
    rPose[t] = move(rPose[t-1],v,mNoiseDist, rng_v, MAP_COLS)

    #make measurement
    zRow, zVal = measure(realLayout,rPose[t],rng_m, valStdErr, rNoiseDist, ptm)

    #update the histogram 
    hist[zRow,rPose[t]] += 1

    for i in range(NPARTICLES):
        if i < NPARTICLES :
            #sample motion model for each particle - motion along column
            pPose[i,POSE] = sMove(pPose[i,POSE],v,pmNoiseDist,rng_pm, MAP_COLS)
        else:  #keep the last particle as the true particle
            pPose[i,POSE] = rPose[t]
        #update log odds
        pPose[i,LOGWT] += logOddsUpdate(zRow,zVal, valStdErr, int(pPose[i,POSE]),pMap[i], ptm,pErr)
        pCntErr[i] = countErr(hist,rPose[t],pMap[i,CNT],int(pPose[i,POSE]))



    #subtract largest log wt to make the wt calculation be numerically stable
    pPose[:,LOGWT] = pPose[:,LOGWT] - np.max(pPose[:,LOGWT])
    pPose[:,WT] = np.exp(pPose[:,LOGWT])
    pPose[:,WT] = pPose[:,WT]/np.sum(pPose[:,WT])         


    if np.min (pPose[:,LOGWT]) < -5 and t < NSTEPS-1: #resample except in the last timestep
        pPose, pMap = sir(pPose,pMap,rngSir)

    #Maximum Likelihood Particle         
    mle = np.argmax(pPose[:,LOGWT],axis=0)
    estMap = np.greater(pMap[mle,LODD],0).astype(int)*pMap[mle,VAL]
    estMap = estMap.astype(int)
    estPose = int(pPose[mle,POSE])
    realPose = rPose[t]
    #mapError[t] = slamError ( estMap, estPose, realLayout, realPose)

#end for i        

print("Histogram of Count Hits")       
print(hist)
print("Histogram of counts from most likely particle")
print(pMap[mle,CNT].astype(int))
print("Error in counts", countErr(hist,int(realPose),pMap[mle,CNT], int(pPose[mle,POSE])))
print("Estimated pose = ", estPose)
print("Estimated Map is:")
print(estMap)
print("Real Pose = ", realPose)
print("Real Map is:")
print(realLayout)



Histogram of Count Hits
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [43.  0.  0. 44.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0. 46.  0. 45.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 48.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 6. 46.  4.  4.  2. 52. 50. 51.  0. 49.]]
Histogram of counts from most likely particle
[[ 0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  1  0  0  0  0  0  0]
 [43  0  0 44  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0]
 [ 0  0 46  0 45  0  0  0  0  0]
 [ 0  0  1  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  0  0  0  0  0 48  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 8 43  6  5  7 51 48 46  5 45]]
Error in counts 38.0
Estimated pose =  3
Estimated Map is:
[[  0   0   0   0   0   0   0   0